# Méthodes d'optimisation - TP 1
## Plus court chemin dans un graphe : Dijkstra et A\*

## Préambule

On va commencer par charger les différentes bibliothèques utilisées dans ce TP :

  - `json` pour lire les fichiers
  - `inf` pour représenter un coût infini ; `pi`, `sin`, `cos`, `acos` pour certains calculs
  - `defaultdict` pour faire des dictionnaires plus économes en mémoire
  - `heapq` pour améliorer nos algorithmes
  - quelques fonctions qui vous sont données (que vous pouvez regarder pour comprendre leur fonctionnement)
  - des informations de typage pour aider l'IDE à vous guider

In [1]:
import json
from math import inf, pi, sin, cos, acos
from collections import defaultdict
import heapq
from utilitaires_tp1 import convertir_graphe, sommet_adresse, visualisation_route, visualisation_rails
from typing import Dict, List, Callable, Tuple

On va ensuite charger les données. Il y aura cinq jeux de données :

  - un petit graphe (celui donné en exemple dans le sujet PDF)
  - un graphe plus grand (300 sommets) avec peu d'arêtes par sommet
  - un autre avec cette fois plus d'arêtes par sommet
  - un grand graphe (le réseau ferré d'Ile-de-France)
  - un très grand graphe (les routes et rues d'Ile-de-France)

In [2]:
# Petit graphe
with open('../data/TP1/petit_graphe.json', 'r') as f:
    petit_graphe = json.load(f)

# Grand graphe (peu d'arêtes)
with open('../data/TP1/graphe_creux.json', 'r') as f:
    graphe_creux = json.load(f)

# Grand graphe (nombreuses arêtes)
with open('../data/TP1/graphe_dense.json', 'r') as f:
    graphe_dense = json.load(f)

# Réseau ferré d'Ile-de-France
with open('../data/TP1/reseau_ferre_idf/graphe.json', 'r') as f:
    graphe_rfidf = json.load(f)
# Pour la visualisation, on va charger deux autres fichiers d'information : les sommets et les gares
with open('../data/TP1/reseau_ferre_idf/sommets.json', 'r') as f:
    sommets_graphe_rfidf = json.load(f)
with open('../data/TP1/reseau_ferre_idf/gares.json', 'r') as f:
    gares_rfidf = json.load(f)

# Grand graphe
with open('../data/TP1/reseau_routier_idf/graphe_idf.json', 'r') as f:
    graphe_idf = json.load(f)
# Pour la visualisation et la recherche, on va charger d'autres informations : les sommets et les adresses connues
with open('../data/TP1/reseau_routier_idf/graphe_idf_sommets.json', 'r') as f:
    sommets_idf = json.load(f)
with open('../data/TP1/reseau_routier_idf/graphe_idf_annuaire.json', 'r') as f:
    annuaire_idf = json.load(f)

#### Remarque sur la structure des graphes dans ce TP

Les graphes donnés sont sous forme de dictionnaires d'adjacence : `graphe[a]` renvoie un dictionnaire des voisins de `a`, et si `b` est un voisin, `graphe[a][b]` renvoie l'arête de `a` vers `b`.

## Une première approche, par programmation dynamique

Voici une première approche qui calcule la distance la plus courte dans le graphe entre **tous** les couples de points. Pour cela, on va considérer les trajets entre deux sommets en autorisant de plus en plus de sommets _intermédiaires_.

On note $d_k(i, j)$ la plus petite distance d'un chemin allant du sommet $i$ au sommet $j$ où **tous** les sommets intermédiaires, où l'on passe, ont un identifiant **strictement inférieur** à $k$. S'il n'existe pas de tel chemin, on a $d_k(i, j) = + \infty$.

Notons que pour $k = 0$, on ne prend en compte que les chemins directs et, en notant $N$ le nombre total de sommets, on cherche à calculer les $d_N(i, j)$.

On admettra les deux propriétés suivantes :
  - Pour tout $i, j, k$, on a $d_{k + 1}(i, j) \leq d_k(i, j)$.
  - Pour tout $i, j, k$, on a $ d_{k + 1}(i, j) = \min\bigl(d_k(i, j), d_k(i, k) + d_k(k, j)\bigr) $.

Nous allons utiliser ces relations pour calculer les $d_k(i, j)$ efficacement, c'est l'algorithme de Floyd-Warshall. Il s'agit d'un algorithme de _programmation dynamique_. On suppose que le graphe est représenté par un dictionnaire d'adjacences

In [3]:
def floyd_warshall(graphe: Dict[str, Dict[str, float | int]]) -> Dict[str, Dict[str, float | int]]:
    # Comme on travaille avec un dictionnaire de dictionnaires, il faut extraire la liste des sommets
    sommets = set()
    for s in graphe.keys():
        sommets.add(s)
        for v in graphe[s].keys():
            sommets.add(v)
    # On crée la matrice à la bonne taille, avec des infinis partout.
    mat = {}
    for i in sommets:
        mat[i] = {}
        for j in sommets:
            mat[i][j] = inf
    # On remplit initialement la matrice avec les poids des arcs existants
    # On met des 0 sur la diagonale pour indiquer que la distance d'un nœud à lui-même est 0
    for i in sommets:
        mat[i][i] = 0
        for j in graphe[i]:
            mat[i][j] = graphe[i][j]
    # On calcule les d_{k + 1}(i, j)
    for k in sommets:
        for i in sommets:
            for j in sommets:
                dij = mat[i][j]
                dik = mat[i][k]
                dkj = mat[k][j]
                mat[i][j] = min(dij, dik + dkj)
    # On a fini
    return mat

Nous allons tester cette fonction sur le petit graphe

In [4]:
fwd = floyd_warshall(petit_graphe)

On vérifie qu'on trouve que le plus court chemin entre A et F est 21 :

In [5]:
assert fwd['A']['F'] == 21

On peut tester sur un plus gros graphe, mais le temps de calcul est rapidement prohibitif. Pour le graphe creux ou le graphe dense, ça prend un petit peu de temps (il y a $300$ sommets, et $300^3 = 27\,000\,000$).


Pour le réseau ferré d'Ile-de-France, il faut compter un peu plus de 2 h de calcul sur une machine puissante (il y a $1689$ sommets, soit plus de $4.8$ milliards d'itérations). Pour le réseau des routes et rues d'Ile-de-France, comptez à peu près $225\,800\,000\,000\,000$ *années*.

In [6]:
fwd_gr_creux = floyd_warshall(graphe_creux)
fwd_gr_dense = floyd_warshall(graphe_dense)

On vérifie que le plus court chemin entre S001 et S300 est 3746 (graphe dense), et qu'entre S011 et S295, il est de 4296 (graphe dense), et qu'entre S006 et S014, il est de 34175 (graphe creux) :

In [7]:
assert fwd_gr_dense['S001']['S300'] == 3746
assert fwd_gr_dense['S011']['S295'] == 4296
assert fwd_gr_creux['S006']['S014'] == 34175

On vérifie aussi que les propriétés attendues sont vérifiées :

  - le coût d'un sommet vers lui-même est nul
  - les coûts sont symétriques (graphe non orienté uniquement)

In [8]:
assert all([fwd_gr_dense[s][s] == 0 for s in fwd_gr_dense.keys()])
assert all([fwd_gr_dense[s1][s2] == fwd_gr_dense[s2][s1] for s1 in fwd_gr_dense.keys() for s2 in fwd_gr_dense.keys()])

## Seconde approche, algorithme de Dijkstra

Nous allons maintenant fixer un sommet de départ (la _source_) et tester tous les plus courts chemins entre celui-ci et tous les autres sommets.

### Rappel

On rappelle la structure générale du parcours d'un graphe :

```python
def parcours_depuis(graphe, source, destination):
    vus = [False] * len(n)
    sac = [source]
    while len(sac) > 0:
        sommet = sac.pop() # on extrait un sommet du sac
        if sommet == destination:
            return True # on est arrivé
        if not vus[sommet]: # si c'est la première fois qu'on le voit
            vus[sommet] = True # on le marque comme vu
            for voisin in graphe[sommet]:
                sac.append(voisin) # on met ses voisins dans le sac
    return False # on n'a pas réussi à atteindre la destination
```

Dans cette approche simple, on indique simplement s'il est possible de relier une _destination_ à partir d'une _source_ donnée.

Différents types de parcours sont obtenus en modifiant le comportement du _sac_, pour lequel on doit pouvoir faire trois choses (en plus de créer un nouveau sac vide) :
1. déterminer s'il est vide ou non,
2. extraire un élément du sac,
3. ajouter un élément dans le sac.

Dans l'exemple précédent, on utilise une liste avec extraction de l'élément le plus à droite et ajout à droite des éléments (c'est une _pile_), cela réalise un parcours _en profondeur_. Si l'on ajoute à droite les éléments et que l'extraction se fait sur l'élément le plus à gauche (c'est une _file_), cela réalise un parcours _en largeur_.

### Algorithme de Dijkstra

Pour obtenir l'algorithme de Dijkstra, l'idée est d'avoir un sac dont tous les éléments ont une valeur associée (ici, la longueur d'un chemin depuis la source) et d'extraire les éléments par distance croissante. Cela assure, lorsque l'on extrait un élément, que le chemin correspondant est le plus court.

Il faut donc être capable de gérer efficacement l'extraction du minimum.

#### Extraction du minimum

Nous allons tout d'abord écrire une fonction `extraire_minimum` qui, comme son nom l'indique, extrait et retourne une valeur minimale du tableau (non vide) passé en argument. Nous allons pour cela procéder en deux temps :
1. déterminer la position du minimum, à l'aide d'une fonction `position_minimum`,
2. puis extraire ce minimum _de façon efficace_, en limitant les manipulations de tableau.

**Exercice**

Écrire une fonction `position_minimum` qui renvoie la position du plus petit élément du tableau non vide passé en argument. Si le plus petit élément apparaît plusieurs fois, on renverra la position de sa première occurrence.

In [9]:
def position_minimum(tab: list) -> int:
    assert len(tab) > 0
    min = 0
    for i in range(len(tab)):
        val = tab[min]
        if (tab[i] < tab[min]):
            min = i
    return min


In [10]:
assert position_minimum([24, 32, 11, 23]) == 2
assert position_minimum([24, 12, 23, 42]) == 1
assert position_minimum([21, 22, 23, 24]) == 0
assert position_minimum([34, 33, 32, 31]) == 3
assert position_minimum([21, 20, 20, 21]) == 1

Pour l'extraction du minimum, il faut tenir compte du fait qu'il est en général couteux de supprimer un élément _au milieu_ d'un tableau, alors que supprimer le dernier élément se fait en temps constant. Nous allons donc procéder ainsi :
1. on récupère la position p du plus petit élément,
2. on stocke la valeur correspondante, c'est ce que nous renverrons à la fin,
3. on met à la position p la dernière valeur du tableau,
4. on supprime la dernière case du tableau à l'aide d'un `tab.pop()`,
5. on renvoie la valeur stockée à l'étape 2.

**Exercice**

Écrire une fonction `extraire_minimum` qui met cette méthode en œuvre.

In [11]:
def extraire_minimum(tab: list) -> int:
    p = position_minimum(tab)
    min = tab[p]
    if p < len(tab) - 1:
        tab[p] = tab.pop()
    else:
        tab.pop()
    return min

In [12]:
tableau = [4, 3, 0, 9, 7, 1, 6, 5, 2, 8]
assert extraire_minimum(tableau) == 0
assert tableau == [4, 3, 8, 9, 7, 1, 6, 5, 2]

#### Organisation du sac

Comme expliqué plus haut, les éléments du sac se verront tous associés une valeur (ici une distance), et le choix de l'élément à extraire du sac se fera en minimisant cette valeur. Pour éviter de modifier la fonction d'extraction du minimum (afin d'indiquer qu'il faut comparer non pas les identifiants, mais la distance associée), on peut utiliser le truc suivant :

<center>
    C'est l'ordre lexicographique qui est utilisé pour comparer des tuples.
</center>

Ainsi, pour comparer deux tuples, on commence par comparer leurs premières composantes, et ce n'est qu'en cas d'égalité que l'on regarde la suite.

In [13]:
tableau = [(4, "a"), (2, "c"), (8, "d"), (5, "b")]
assert extraire_minimum(tableau) == (2, "c")

Ainsi, pour l'algorithme de Dijkstra, il suffit de mettre dans les sacs des couples `(distance, sommet)`.
Notons que lors de l'extraction d'un élément, on pourra écrire, par exemple :
```python
distance, sommet = extraire_minimum(sac)
```

**Exercice**
Écrire la fonction `dijkstra` implémentant l'algorithme de Dijkstra.

In [14]:
def dijkstra(graphe: Dict[str, Dict[str, float | int]], src: str, dst: str) -> float | int:
    # Comme on travaille avec un dictionnaire de dictionnaires, il faut extraire la liste des sommets
    sommets = set()
    for s in graphe.keys():
        sommets.add(s)
        for v in graphe[s].keys():
            sommets.add(v)

    # On initialise le dictionnaire des sommets vus
    vus = dict()
    for s in sommets : vus[s] = False

    # On initialise le sac
    sac = [(0, src)]  # (distance, sommet)

    # On cherche le plus court chemin
    distances = {s: inf for s in sommets}
    distances[src] = 0

    while len(sac) > 0:
        # On extrait le chemin de distance minimale
        (chemin, smt) = extraire_minimum(sac)

        # On a trouvé le sommet de destination
        if smt == dst: return chemin
        
        # On marque le sommet comme vu
        if vus[smt]: continue
        else: vus[smt] = True

        # On met à jour les distances des voisins
        for voisin, poids in graphe[smt].items():
            if not vus[voisin]:
                # On calcule la nouvelle distance
                nouvelle_distance = chemin + poids
                
                # On met à jour le sac
                distances[voisin] = nouvelle_distance
                sac.append((nouvelle_distance, voisin))

    return -1  # On n'a pas trouvé de chemin

In [15]:
assert dijkstra(petit_graphe, 'A', 'F') == 21
assert dijkstra(graphe_creux, 'S001', 'S300') == 8302
assert dijkstra(graphe_dense, 'S001', 'S300') == 3746

### Une première amélioration

Sur un graphe dense, on peut voir que cette implémentation est assez lente. Par exemple, pour aller de S001 à S300, l'algorithme va examiner dans l'ordre de distance tous les sommets jusqu'à atteindre la destination, et surtout cela va entraîner l'examen d'environ 37000 arêtes.

**Question**

Pouvez-vous retrouver ce nombre d'arêtes en modifiant un peu l'algorithme de Dijkstra précédent ? Il est **vivement** recommandé de faire une copie de la fonction dans une nouvelle cellule (voir le menu `Insérer` ci-dessus) avant de faire des modifications.

In [ ]:
def dijkstra(graphe: Dict[str, Dict[str, float | int]], src: str, dst: str, compte=False) -> float | int:
    # Comme on travaille avec un dictionnaire de dictionnaires, il faut extraire la liste des sommets
    sommets = set()
    for s in graphe.keys():
        sommets.add(s)
        for v in graphe[s].keys():
            sommets.add(v)
    # On initialise notre compteur
    cpt = 0
    
    # On initialise le dictionnaire des sommets vus
    vus = dict()
    for s in sommets : vus[s] = False
    
    # On initialise le sac
    sac = [(0, src)]  # (distance, sommet)
    
    # On cherche le plus court chemin
    distances = {s: inf for s in sommets}
    distances[src] = 0

    while len(sac) > 0:
        # On extrait le chemin de distance minimale
        (chemin, smt) = extraire_minimum(sac)

        # On a trouvé le sommet de destination
        if smt == dst:
            print(f"Nombre d'arêtes explorées : {cpt}")
            return chemin
        
        # On marque le sommet comme vu
        if vus[smt]: continue
        else: vus[smt] = True

        # On met à jour les distances des voisins
        for voisin, poids in graphe[smt].items():
            if not vus[voisin]:
                # On calcule la nouvelle distance
                nouvelle_distance = chemin + poids
                
                # On met à jour le sac
                distances[voisin] = nouvelle_distance
                sac.append((nouvelle_distance, voisin))
                cpt += 1

    return -1  # On n'a pas trouvé de chemin

In [30]:
# Doit afficher "Nombre d'arêtes explorées : <nb>"
dijkstra(graphe_dense, 'S001', 'S300', True)

Nombre d'itérations : 23296


3746

La raison est que la fonction de recherche de la position du minimum dans une liste se fait en temps linéaire et l'exemple ci-dessus montre les limites d'une telle complexité.

Il existe des manières d'organiser les données pour que la recherche et l'extraction du minimum soit plus efficace, c'est ce que l'on appelle une _file de priorité_. Sans entrer dans les détails, on peut faire en sorte que l'extraction du minimum et l'ajout d'une valeur se fassent en temps _logarithmique_ et non plus linéaire. Cela donne un _sac_ bien plus efficace.

Pour voir cela, on peut utiliser la bibliothèque [`heapq`](https://docs.python.org/3/library/heapq.html) de Python. On crée un sac vide à l'aide d'une liste vide, comme précédemment. Les modifications à effectuer sont :
1. pour extraire le minimum du sac, on écrit `heapq.heappop(sac)`,
2. pour ajouter un élément `elt` au sac, on écrit `heapq.heappush(sac, elt)`.


**Exercice**

Reprendre votre fonction _dijkstra_ et la modifier pour utiliser une file de priorité du module `heapq`. On va également remplacer le dictionnaire des sommets vus par un `defaultdict`, ce qui permet de se passer d'énumérer les sommets à l'initialisation.

In [32]:
def dijkstra_rapide(graphe: Dict[str, Dict[str, float | int]], src: str, dst: str, compte=False) -> float | int:
    # On initialise avec un defaultdict
    vus = defaultdict(lambda: False)

    # On initialise le sac
    sac = []

    # On garde la possibilité de compter les arêtes
    it = 0

    # On utilise heapq pour le sac à priorité
    heapq.heappush(sac, (0, src))

    # Puis, on procède de manière similaire, avec heapq au lieu de nos listes simples et retrait d'élément manuels
    vus[src] = 0

    while len(sac) > 0:
        # On extrait le chemin de distance minimale
        (chemin, smt) = heapq.heappop(sac)

        # On a trouvé le sommet de destination
        if smt == dst:
            print(f"Nombre d'arêtes explorées : {it}")
            return chemin
        
        # On marque le sommet comme vu
        if vus[smt]: continue
        else: vus[smt] = True

        # On met à jour les distances des voisins
        for voisin, poids in graphe[smt].items():
            if not vus[voisin]:
                # On calcule la nouvelle distance
                nouvelle_distance = chemin + poids
                
                # On met à jour le sac
                heapq.heappush(sac, (nouvelle_distance, voisin))
                it += 1

    return -1  # On n'a pas trouvé de chemin
    

In [33]:
assert dijkstra_rapide(graphe_dense, 'S001', 'S300', True) == 3746

Nombre d'arêtes explorées : 23296


#### Données réelles : le réseau ferré d'IdF et le réseau routier d'IdF

Nous allons maintenant regarder deux réseaux réels :
  - le premier est celui du réseau ferré en Ile-de-France (trains, métro, trams, TER, grandes lignes, orlyval...), il comporte 1689 sommets (gares et fourches sur les voies) et 1930 arêtes.
  - le second est celui du réseau routier en Ile-De-France. Il y a 233223 sommets (intersection entre au moins deux chemins/rues/routes ou la fin d'une impasse) et 256586 arêtes.

Pour ces deux graphes, nous avons deux poids par arêtes : le temps et la distance, pour les extraire, nous avons un utilitaire : `convertir_graphe` afin d'obtenir nos structures habituelles.

In [ ]:
graphe_ferroviaire_temps = convertir_graphe(graphe_rfidf, 'temps')
graphe_ferroviaire_distance = convertir_graphe(graphe_rfidf, 'distance')
graphe_routier_temps = convertir_graphe(graphe_idf, 'temps')
graphe_routier_distance = convertir_graphe(graphe_idf, 'distance')

Dans ces fichiers, les sommets sont identifiés par un code (une chaine de caractères) unique qui ne donne pas d'information sur ce qu'il représente réellement. Nous allons utiliser des fonctions qui vont nous aider à travailler avec les données que nous connaissons (nom de gares, noms de villes/rues).

Pour le réseau ferroviaire, on a deux dictionnaires :
  - `sommets_graphe_rfidf` dit pour chaque sommet ses coordonnées géographique (\[longitude, latitude\]) et l'identifiant de la gare qui lui correspond quand il y en a une
  - `gares_rfidf` dit pour chaque identifiant de gare son nom (avec les informations affichées au-dessus/au-dessous sur les panneaux), sa ligne et l'exploitant. Attention, une gare au sens physique a autant de sommets qu'il y a de lignes. Ainsi, la gare `Bastille` a trois identifiants, et donc trois sommets.

**Exercice**

Écrire une fonction `id_gare` qui étant donné un dictionnaire de gares et nom de gare, renvoie le ou les identifiants correspondants (et une liste vide si la gare n'est pas trouvée).

In [34]:
def id_gare(dico_gares: Dict[str, Dict[str, any]], nom_gare: str) -> List[str]:
    identifiants = []
    
    for id_gare, info in dico_gares.items():
        if info['nom'].lower() == nom_gare.lower():
            identifiants.append(id_gare)

    return identifiants

In [35]:
assert id_gare(gares_rfidf, 'Bastille') == ['325', '1172', '324']
assert id_gare(gares_rfidf, 'Pastille') == []
assert id_gare(gares_rfidf, 'Le Guichet') == ['31']

**Exercice**

Écrire une fonction `sommet_gare` qui, étant donné un dictionnaire de sommets, trouve le sommet correspondant à un identifiant donné, et `None` sinon.

In [ ]:
def sommet_gare(dico_sommets: Dict[str, Dict[str, any]], code_gare: str) -> None|str:
    ...
    return None

In [ ]:
assert sommet_gare(sommets_graphe_rfidf, '325') == 'S000915'
assert sommet_gare(sommets_graphe_rfidf, '2325') is None

**Exercice**

Écrire la fonction `gare_sommet` qui fait le travail inverse : avec le code du sommet, renvoyer les informations sur la gare (informations depuis le dictionnaire des gares plus ses coordonnées géographiques).

In [ ]:
def gare_sommet(dico_gares: Dict[str, Dict[str, any]], dico_sommets: Dict[str, Dict[str, any]], sommet: str) -> None | Dict[str, None|str|List[int|float]]:
    ...

In [ ]:
assert gare_sommet(gares_rfidf, sommets_graphe_rfidf, 'S000915') == {'nom': 'Bastille', 'nom_sous': None,
                                                                     'nom_sur': None, 'ligne': 'METRO 8',
                                                                     'exploitant': 'RATP',
                                                                     'coordonnees': [2.369513568156525,
                                                                                     48.85343942664758]}
assert gare_sommet(gares_rfidf, sommets_graphe_rfidf, 'S999915') is None

Les distances sont en mètres et les temps en secondes (c'est très approximatif) pour le réseau ferroviaire. Quel est le plus court chemin en temps et en distance entre `Le Guichet` et `Vélizy 2` ?

In [ ]:
gare_depart = 'Le Guichet'
gare_arrivee = 'Vélizy 2'
...
print(f'Temps entre {gare_depart} et {gare_arrivee} : ',
      ..., 'minutes')
print(f'Distance entre {gare_depart} et {gare_arrivee} : ',
      ..., 'km')

On vous donne une fonction `sommet_adresse` qui prend en entrée un annuaire et un dictionnaire représentant l'adresse et qui vous renvoie un dictionnaire de toutes les adresses trouvées avec le sommet correspondant.

Exemples :

In [ ]:
print(sommet_adresse(annuaire_idf, {'ville': 'Gif-sur-Yvette', 'rue': 'Avenue des Sciences'}))
print(sommet_adresse(annuaire_idf, {'ville': 'Paris', 'rue': 'Rue de Grenelle'}))

Avec ces adresses, on trouve ainsi :

In [ ]:
print(round(dijkstra_rapide(graphe_routier_distance, '25580313', '1442397072') / 1000, 3), 'km')
print(round(dijkstra_rapide(graphe_routier_temps, '25580313', '1442397072') / 60), 'minutes')

On peut aussi tester sur le réseau ferré, et vérifier combien d'arêtes sont visitées :

In [ ]:
print(dijkstra_rapide(graphe_ferroviaire_temps, 'S000110', 'S000590', True) / 60, 'minutes')

<details>
    <summary>Remarque de complexité</summary>
    <blockquote>
Avec une fonction d'extraction de minimum, on peut imager un algorithme de tri : on ajoute un à un tous les éléments d'une liste, puis on les extrait un par un en prenant à chaque fois le minimum.

La méthode naïve d'extraction du minimum, de complexité linéaire, conduit à un algorithme de tri de complexité quadratique, comme le tri par sélection ou le tri par insertion.

L'usage d'une file de priorité plus performante, de complexité linéaire, conduit à un algorithme de tri de complexité optimale, en <i>n</i> ln(<i>n</i>). C'est ce que l'on appelle le <i>tri par tas</i>.

Notons que c'est un algorithme de tri de complexité optimale, ce qui signifie que l'on ne peut pas faire mieux qu'une complexité logarithme en moyenne pour traiter un élément dans une file de priorité.
    </blockquote>
</details>

## Une seconde amélioration, l'algorithme A*


Dans certaines situations, on peut grandement améliorer l'efficacité de l'algorithme de Dijkstra pour calculer la longueur du plus court chemin entre deux points, en ajoutant à la distance des sommets du sac une autre valeur, une _heuristique_ qui va modifier l'ordre d'évaluation des sommets.

Un cadre où cela fonctionne particulièrement bien est lorsque l'on est dans un contexte géométrique, que les sommets sont des points et les poids des arêtes sont des distances. On peut alors utiliser comme heuristique la distance d'un sommet au sommet destination.

Ainsi, pour chaque élément du sac, en plus du sommet en question et de la distance (qui correspond à la longueur d'un chemin de la source jusqu'à ce sommet), on va utiliser cette distance augmentée de la distance du sommet à la destination, et c'est cette distance augmentée que l'on va prendre en compte pour extraire le minimum.

Intuitivement, cela revient à consider les chemins par "distance croissante" par rapport à la trajectoire directe, _à vol d'oiseau_. Si le chemin n'est pas trop compliqué, l'algorithme le trouvera très vite (mais il n'est pas magique, et dans un contexte type labyrinthe, A* risque d'être moins efficace (ne pas trouver le chemin le plus court) que la version de base de l'algorithme de Dijkstra). Pour aider, nous allons modifier notre algorithme : désormais, on s'autorisera à revisiter un sommet pour mettre à jour sa distance au point de départ si on l'améliore.

<br/>

Dans notre approche, nous allons devoir calculer la distance entre deux points de la surface du globe, que l'on appelle _distance orthodromique_.

On donne la fonction suivante, qui calcule cette distance orthodromique entre deux points du globe spécifiés par leurs coordonnées en degrés.

In [ ]:
def distance_orthodromique(lat1: float, lng1: float, lat2: float, lng2: float) -> float:
    # angles en degrés
    lat1 *= pi / 180
    lng1 *= pi / 180
    lat2 *= pi / 180
    lng2 *= pi / 180
    v = sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng1 - lng2)
    # Gestion des dépassements de flottants
    if v > 1:
        v = 1
    elif v < -1:
        v = -1
    return 6371000 * acos(v)

On vous donne la fonction `distance_sommets` qui calcule la distance orthodromique entre les sommets dont on a précisé les identifiants. On rappelle que la latitude et la longitude d'un sommet figurent parmi les informations présentes dans le dictionnaire `sommets_idf`.

In [ ]:
def distance_sommets_routes(id1: str, id2:str) -> float:
    s1 = sommets_idf[id1]
    s2 = sommets_idf[id2]
    return distance_orthodromique(s1['latitude'], s1['longitude'], s2['latitude'], s2['longitude'])

De même pour les gares et le dictionnaire `sommets_graphe_rfidf`:

In [ ]:
def distance_sommets_gares(id1: str, id2: str) -> float:
    s1 = sommets_graphe_rfidf[id1]['coordonnees']
    s2 = sommets_graphe_rfidf[id2]['coordonnees']
    return distance_orthodromique(s1[1], s1[0], s2[1], s2[0])

Comme indiqué précédemment, dans le sac, nous allons devoir stocker, en plus de la donnée d'un sommet et d'une distance (correspondant à la longueur d'un chemin de la source à ce sommet), la distance augmentée de la distance orthodromique de ce sommet à la destination. Les sacs devront donc contenir des triplets, constitués de telle sorte que c'est la distance augmentée qui est utilisée pour l'extraction du minimum.

**Exercice**

Mettre en œuvre l'algorithme A* en reprenant et modifiant l'algorithme de Dijkstra précédent. Pour gérer le calcul de distance de manière flexible, votre fonction utilisera un appel à `heuristique(sommet1, sommet2)` avec `heuristique` la fonction passée en paramètre à A\*. Exemple d'appel de `astar`: `astar(mon_graphe, source, destination, distance_sommets_routes)`.

In [ ]:
def astar(graphe: Dict[str, Dict[str, float | int]], src: str, dst: str, heuristique: Callable[[str, str], float|int], compte=False) -> int|float:
    # Initialisation : comme Dijkstra
    vus = ...
    sac = ...
    it = 0
    ...
    # Résolution : il faut maintenant utiliser l'heuristique comme pour calculer la priorité
    ...

**Exercice**

Comparez l'efficacité de l'algorithme A* avec celui de Dijkstra en comptant dans chaque cas le nombre d'arêtes et de sommets traités sur un long trajet, par exemple de la Rue de Grenelle (25580313) à l'Avenue des Sciences (1442397072) comme ci-dessus.

In [ ]:
d1 = dijkstra_rapide(graphe_routier_distance, '21378248', '1442397072', True)
d2 = astar(graphe_routier_distance, '21378248', '1442397072', distance_sommets_routes, True)
print(d1, d2)

**Question**

Combien d'arêtes l'algorithme A* va-t-il traiter s'il existe un trajet direct (sans sommet intermédiaire) entre deux destinations ?

#### Limites de l'algorithme A\*

Regardons maintenant le cas de la recherche du plus court chemin *en temps*. En prenant les mêmes sommets et la même heuristique, que donnent les deux algorithmes Dijkstra et A\* sur le graphe du réseau routier en temps ?

Que constatez-vous ? Comment expliquez-vous ce résultat ?

**Exercice**

Proposez une autre heuristique pour estimer le *temps à vol d'oiseau*

In [ ]:
d1 = dijkstra_rapide(graphe_routier_temps, '21378248', '1442397072', True)
d2 = astar(graphe_routier_temps, '21378248', '1442397072', distance_sommets_routes, True)
print(d1, d2)

**Exercice**

Modifier les heuristiques de distance pour adapter au temps.

In [ ]:
def temps_sommets_routes(id1: str, id2:str) -> float:
    ...

def temps_sommets_gares(id1: str, id2:str) -> float:
    ...

## Récupération du trajet le plus court et visualisation

Une fois que l'on a déterminé le plus court chemin entre deux gares ou adresses, encore faut-il pouvoir récupérer la liste des sommets de ce trajet le plus court. Cela se fait très simplement, en indiquant pour chaque sommet, d'où l'on vient lorsqu'on l'a découvert.

Une façon de faire cela suppose deux modifications de l'algorithme précédent (Dijkstra ou A*) :
1. le tableau `vus`, plutôt que de contenir une valeur booléenne, contiendra des entiers, avec une valeur `None` à l'initialisation et tant que l'on n'a pas découvert le sommet et une valeur autre si le sommet a déjà été découvert, la valeur étant alors l'identifiant du sommet d'où l'on venait lors de la découverte.
2. le sac contiendra des _tuples_ avec une composante supplémentaire, correspondant au sommet de provenance.

**Exercice**

Modifier l'algorithme précédent pour traiter les informations de provenance pour chaque gare/sommet.

In [ ]:
def dijkstra_rapide_chemin(graphe: Dict[str, Dict[str, float | int]], src: str, dst: str, compte=False) -> Tuple[int|float, Dict[str, str]]:
    ...

In [ ]:
def astar_chemin(graphe: Dict[str, Dict[str, float | int]], src: str, dst: str, heuristique: Callable[[str, str], float|int], compte=False) -> Tuple[int|float, Dict[str, str]]:
    ...

#### Reconstitution du chemin

Faire ensuite une fonction `reconstruire_chemin` qui prend le dictionnaire reçu en entrée, une source et une destination et renvoie une liste de nœuds dans le bon ordre.

In [ ]:
def reconstruire_chemin(parents: Dict[str, str], src: str, dest: str) -> List[str]:
    ...

### Visualisation

Dans les utilitaires fournis pour ce TP, il y a une fonction qui à partir du graphe complet (avec les données géographiques, le chemin et le départ/arrivée, va vous générer une carte qui peut être affichée dans le classeur. Celle-ci peut également prendre un second chemin de même départ/arrivée pour comparer deux algorithmes.

In [ ]:
depart = '25580313'
arrivee = '1442397072'
_, ch = astar_chemin(graphe_routier_distance, depart, arrivee, distance_sommets_routes)
chemin_plus_court = reconstruire_chemin(ch, depart, arrivee)
_, ch = astar_chemin(graphe_routier_temps, depart, arrivee, temps_sommets_routes)
chemin_plus_rapide = reconstruire_chemin(ch, depart, arrivee)
visualisation_route(graphe_idf, sommets_idf, chemin_plus_court, chemin_plus_rapide)

Faites de même pour un trajet sur le réseau ferré.

In [ ]:
gare_depart = 'Le Guichet'
gare_arrivee = 'Vélizy 2'
...